In [1]:
import os
import pickle

In [2]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/entbappy/End-to-end-Machine-Learning-Project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    metric_file_name: Path
    mlflow_uri: str
    transformed_data_test: Path
    transformed_data_train: Path
    model_path: Path
    all_params: dict
    target_column: str


In [7]:
from RegressionProject.constants import *
from RegressionProject.utils.common import read_yaml, create_directories, save_json, load_object_pkl

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config_model_eval = self.config.model_evaluation
        config_model_train = self.config.model_trainer
        config_model_transform = self.config.data_transformation

        params = self.params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config_model_eval.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config_model_eval.root_dir,
            metric_file_name = config_model_eval.metric_file_name,
            mlflow_uri=config_model_eval.mlflow_uri,
            transformed_data_test=config_model_transform.transformed_data_test,
            transformed_data_train=config_model_transform.transformed_data_train,
            model_path = config_model_train.trained_model_file_path,
            target_column = schema.name,

            all_params=params,
           
        )
        print(model_evaluation_config)
        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np

In [14]:
from RegressionProject.utils.common import read_transformed_data
import pickle
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        _, _, test_x, test_y = read_transformed_data(self.config.transformed_data_train, self.config.transformed_data_test)

        # Load the model from the pickle file
        model=load_object_pkl(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_math_scores = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_math_scores)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="best_model")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-27 10:36:57,842: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 10:36:57,845: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 10:36:57,847: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-27 10:36:57,847: INFO: common: created directory at: artifacts]
[2024-06-27 10:36:57,848: INFO: common: created directory at: artifacts/model_evaluation]
ModelEvaluationConfig(root_dir='artifacts/model_evaluation', metric_file_name='artifacts/model_evaluation/metrics.json', mlflow_uri='https://dagshub.com/iheb.aamrii/Mlops-Regression-Project.mlflow', transformed_data_test='artifacts/data_transformation/transformed_test_data.csv', transformed_data_train='artifacts/data_transformation/transformed_train_data.csv', model_path='artifacts/model_trainer/best_model.pkl', all_params=ConfigBox({'Decision_Tree': {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']}, 'Random_Forest': {'n_estimators': [8, 16, 32

Successfully registered model 'best_model'.
2024/06/27 10:37:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best_model, version 1
Created version '1' of model 'best_model'.


In [12]:
import dagshub
dagshub.init(repo_owner='iheb.aamrii', repo_name='Mlops-Regression-Project', mlflow=True)


[2024-06-27 10:35:44,053: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]
[2024-06-27 10:35:44,678: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/iheb.aamrii/Mlops-Regression-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "iheb.aamrii/Mlops-Regression-Project"

[2024-06-27 10:35:44,684: INFO: helpers: Initialized MLflow to track repo "iheb.aamrii/Mlops-Regression-Project"]


Repository iheb.aamrii/Mlops-Regression-Project initialized!

[2024-06-27 10:35:44,686: INFO: helpers: Repository iheb.aamrii/Mlops-Regression-Project initialized!]
